In [7]:
from IPython.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))

import mimikit as mmk
import ipywidgets as W
import dataclasses as dtc
from typing import Optional, Callable, Any, Tuple, List, Dict
from IPython.display import display
UI = mmk.ui

data_cfg = mmk.DatasetConfig()
tr_cfg = mmk.TrainARMConfig()
net_cfg = mmk.WaveNet.Config(io_spec=mmk.WaveNet.qx_io)

class DataView(W.Tab):
    
    def __init__(self, data_config):
        super().__init__(children=[mmk.views.dataset_view(data_config), 
                                   W.VBox(children=[
                                       W.HTML(value="<h4>Select Dataset File</h4>"),
                                       UI.DatasetPicker().widget])])
        self.set_title(0, "New Dataset from Soundfiles")
        self.set_title(1, "Load Dataset File from Disk")

def run(ev):
    print(ev)
    net = mmk.WaveNet.from_config(net_cfg)
    dataset_cls = net.config.io_spec.dataset_class()
    dataset_cls.create(data_cfg.destination, data_cfg.sources, mode="w")
    db = dataset_cls(data_cfg.destination, mode='r', keep_open=True)
    hp = mmk.ARMHP(data_cfg, net_cfg, tr_cfg)
    loop = mmk.TrainLoop.from_config(hp, db, net)
    loop.run()
    
submit = W.Button(description="RUN", layout=W.Layout(width="100%"))
submit.on_click(run)

container = W.VBox([
    DataView(data_cfg),
    mmk.views.wavenet_view(net_cfg),
    mmk.views.train_arm_view(tr_cfg),
    submit
])

display(UI.MMK_STYLE_SHEET, container)

HTML(value='\n<style>\n\n.picker-button {\n    border: 0px !important;\n    border-radius: 5px !important;\n  …

Button(description='RUN', layout=Layout(width='100%'), style=ButtonStyle())


AttributeError: object of type Proxy has no attribute 'shape'

In [17]:



train_view = mmk.ConfigView(
    cfg,
    # ---------------------------
    UI.Param(
        name="data",
        widget=),
    # ---------------------------
    UI.Param(
        name="network",
        widget=),
    # ---------------------------
    UI.Param(
        name="training",
        widget=)
    # Layout
    # Config Validation
)



display(*train_view.widgets)

SyntaxError: invalid syntax (<ipython-input-17-8c80a16fa9b4>, line 6)

In [ ]:
mmk.demos.freqnet.main(cfg)